In [2]:
import pdb;
import os
import re
import time
import torch
import PIL.Image as Image

from transformers import Qwen2VLForConditionalGeneration, Qwen2VLProcessor
# torch.backends.cuda.enable_mem_efficient_sdp(False)
# torch.backends.cuda.enable_flash_sdp(False)

## Resize image using PIL

In [3]:

# img = Image.open('chrome.png', 'r')
# w, h = img.size 
# ratio = w / h
# new_h = 720
# new_w = round(720 * ratio)
# print(new_w, new_h, new_w * new_h // 28 // 28)
# img_resize = img.resize((new_w, new_h))
# img_resize.save('chrome_resized.png')

## Loading model

In [4]:
from qwen_vl_utils import process_vision_info
# from transformers import Qwen2VLForConditionalGeneration, Qwen2VLProcessor

# from model.showui.modeling_showui import ShowUIForConditionalGeneration
# from model.showui.processing_showui import ShowUIProcessor

### ShowUI Preprocessor
# 0. Common setups
min_pixels = 256*28*28
max_pixels = 1344*28*28
# 1. Screenshot -> Graph
uigraph_train = True        # Enable ui graph during training
uigraph_test = True         # Enable ui graph during inference
uigraph_diff = 1            # Pixel difference used for constructing ui graph
uigraph_rand = False        # Enable random graph construction 
# 2. Graph -> Mask 
uimask_pre = True           # Prebuild patch selection mask in the preprocessor (not in model layers) for efficiency
uimask_ratio = 0.5          # Specify the percentage of patch tokens to skip per component
uimask_rand = False         # Enable random token selection instead of uniform selection

### ShowUI Model
lm_skip_ratio = uimask_ratio # valid if not uimask_pre
lm_skip_layer = "[1,28,1]"   # [1,28,1] means we apply UI guide token selection from 1-th to 28-th layer (28 is the last layer of Qwen2-VL)
model_path = "/data/data1/syc/intern/wanshan/models/Qwen2-VL-7B-Instruct"

processor = Qwen2VLProcessor.from_pretrained(
    model_path, 
    min_pixels=min_pixels, max_pixels=max_pixels,
    # uigraph_train=uigraph_train, uigraph_test=uigraph_test, uigraph_diff=uigraph_diff, uigraph_rand=uigraph_rand,
    # uimask_pre=True, uimask_ratio=uimask_ratio, uimask_rand=uimask_rand,
)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [5]:
from transformers import Qwen2VLConfig
config = Qwen2VLConfig.from_pretrained(model_path)
print(config._attn_implementation) # only eager attention implementation can output attention

eager


In [6]:
# lm_qwen_layer = 28 # LLM Decoder layers
device = 'cuda'
# def parse_layer_type(str_ranges, L=lm_qwen_layer, default=0):
#     # 0 is without layer token selection, 1 is with layer token selection. Below we provide examples:
#     # [1,28,1] means that all LM layers use token selection; [1,28,0] means that do not.
#     # Interleaved layer-wise '[2,2,1],[4,4,1],[6,6,1],[8,8,1],[10,10,1],[12,12,1],[14,14,1],[16,16,1],[18,18,1],[20,20,1],[22,22,1],[24,24,1],[26,26,1]'
#     result = [default] * L
#     matches = re.findall(r'\[\s*(\d+)\s*,\s*(\d+)\s*,\s*(\d+)\s*\]', str_ranges)
#     for start, end, value in matches:
#         start, end, value = int(start) - 1, int(end) - 1, int(value)
#         if end >= L:
#             end = L - 1
#         result[start:end + 1] = [value] * (end - start + 1)
#     return result

# lm_skip_layer = parse_layer_type(lm_skip_layer, 28)
# print(lm_skip_layer)


model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map=device,
    # lm_skip_ratio=lm_skip_ratio, lm_skip_layer=lm_skip_layer,
    attn_implementation="eager"
)


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
# print(model)

In [8]:
img_url = 'chrome_resized.png'
vis_dir = 'visualize_imgs'


messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Please describe this image in detail. Explain what is depicted, including all visible components, structures, or elements, "
            "and how they interact with each other. Clarify the purpose of this object or system and the context in which it is typically used. "
            "Additionally, explain why this is important — what problems it helps solve, what needs it fulfills, or what advantages it provides. "
            "If applicable, describe the technology behind it, its functionality, and its role in broader applications. "
            "Your response should be approximately 100 words, providing a comprehensive and informative explanation for better understanding."},
            {
                "type": "image",
                "image": img_url,
                "min_pixels": min_pixels,
                "max_pixels": max_pixels,
            },
        ],
    }
]

# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True,
)
print(text)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)

inputs = inputs.to(device)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Please describe this image in detail. Explain what is depicted, including all visible components, structures, or elements, and how they interact with each other. Clarify the purpose of this object or system and the context in which it is typically used. Additionally, explain why this is important — what problems it helps solve, what needs it fulfills, or what advantages it provides. If applicable, describe the technology behind it, its functionality, and its role in broader applications. Your response should be approximately 100 words, providing a comprehensive and informative explanation for better understanding.<|vision_start|><|image_pad|><|vision_end|><|im_end|>
<|im_start|>assistant



In [9]:
for key in inputs.keys():
    print(key, inputs[key].shape)

input_ids torch.Size([1, 1202])
attention_mask torch.Size([1, 1202])
pixel_values torch.Size([4264, 1176])
image_grid_thw torch.Size([1, 3])


In [10]:
# print(inputs["select_mask"].sum(), inputs['select_mask'].shape)

In [11]:
config.vision_end_token_id

151653

In [13]:
# text_inputs['patch_pos'] = torch.zeros_like(text_inputs['input_ids']) -1
vision_idx = {
    'start': 0,
    'end': 0
}

for i in range(len(inputs['input_ids'][0])):
    # assume here is 1 x L
    if inputs['input_ids'][0, i] == config.vision_start_token_id:  
        vision_idx['start'] = i+1
        print(f'<vision_state> at {i}') # vision_start, image_tokens ... , vision_end
    if inputs['input_ids'][0, i] == config.vision_end_token_id:  
        print(f'<vision_end> at {i}')
        vision_end = i
        vision_idx['end'] = i

vision_idx

<vision_state> at 129
<vision_end> at 1196


{'start': 130, 'end': 1196}

In [14]:
generated_ids = model.generate(
                    **inputs,
                    do_sample=False,
                    max_new_tokens=128,
                    use_cache=True,
                    # stopping_criteria=[stopping_criteria],
                    output_attentions=True,
                    output_scores=True,
                    return_dict_in_generate=True,
                    )
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids['sequences'])
]
output_text = processor.batch_decode(
    generated_ids_trimmed, 
)[0]
print(output_text)
for key in generated_ids.keys():
    print(key, type(generated_ids[key]))

/home/syc/anaconda3/envs/qwen2vl/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.01` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/syc/anaconda3/envs/qwen2vl/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.001` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/syc/anaconda3/envs/qwen2vl/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:651: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


The image depicts the Google Chrome web browser interface. The browser window is open, displaying the Google search homepage. The search bar is prominently displayed in the center, with the Google logo above it. Below the search bar, there are options to add shortcuts for Baidu and other a button to add a new shortcut. The browser also shows various tabs and bookmarks on the top right corner, including Gmail, Images, and All Bookmarks. The bottom of the screen shows the taskbar with various icons, including the Google Chrome icon. The system bar at the bottom of the screen shows the current time and date, as well as the system bar
sequences <class 'torch.Tensor'>
scores <class 'tuple'>
attentions <class 'tuple'>
past_key_values <class 'transformers.cache_utils.DynamicCache'>


In [15]:
inputs['input_ids'].shape

torch.Size([1, 1202])

In [16]:
output_seqs = generated_ids['sequences']
output_seqs.shape

torch.Size([1, 1330])

In [17]:
attention_outputs = generated_ids['attentions']
print(len(attention_outputs)) # [seq_len, ...]

128


In [18]:
len(attention_outputs[0]) # 28 decoder layer 

28

In [19]:
attention_outputs[0][0].shape

torch.Size([1, 28, 1202, 1202])

In [20]:
attention_outputs[0][27].shape

torch.Size([1, 28, 1202, 1202])

In [21]:
attention_outputs[1][0].shape

torch.Size([1, 28, 1, 1203])

In [22]:
scores = generated_ids['scores']
print(len(scores[0]))

1


In [23]:
scores[0].shape

torch.Size([1, 152064])

In [24]:
# import torch
# import torch.nn.functional as F

# # Suppose:
# # C = number of input tokens (e.g., 516)
# # G = number of generated tokens you want to combine (64)
# C = 516
# G = 64
# final_seq_len = C + G

# # full_attention_outputs will be a list with one tensor per layer (28 in total)
# full_attention_outputs = []  # one per decoder layer

# # Loop over each layer (assuming 28 layers)
# for layer in range(1):
#     # Get the cached attention for the input tokens.
#     # This tensor has shape [1, 12, C, C].
#     cached_attn = attention_outputs[0][layer]   # shape: [1, 12, C, C]
#     print(cached_attn.shape)
    
#     # This will store the new token rows (one per generation step).
#     new_tokens_attn = []
    
#     # For generation steps 1 to G (i.e., the first 64 generated tokens)
#     for gen_idx in range(1, G + 1):
#         # Get the new token's attention for the layer.
#         # It should have shape [1, 12, 1, (C + gen_idx)].
#         new_attn = attention_outputs[gen_idx][layer]
#         print(new_attn.shape)
#         cached_attn = torch.cat((cached_attn,new_tokens_attn), dim=2)
#         print(cached_attn.shape)


In [25]:
for key in generated_ids.keys():
    print(key)

sequences
scores
attentions
past_key_values


In [26]:
vision_idx

{'start': 130, 'end': 1196}

In [27]:
generated_ids[0].shape

torch.Size([1, 1330])

In [28]:
generated_ids[0][-1]

tensor([151644,   8948,    198,  ...,    279,   1849,   3619], device='cuda:0')

In [29]:
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import numpy as np
from matplotlib.colors import LogNorm, ListedColormap
import matplotlib.patches as mpatches

def visualize_attention(multihead_attention, 
                        output_path="atten_map_1.png",
                        title="Layer 5", 
                        vision_idx=None, 
                        show_image=False,
                        pool_stride=20, 
                        dpi=200):
    """
    Visualize a single attention map without tick labels on the main heatmap.
    Adds thin colored strips along the LEFT and BOTTOM edges to indicate token regions:
      - Blue: Text tokens (System+Instruction) outside vision token range
      - Red: Vision tokens (vision_idx['start'] to vision_idx['end'])
    Additionally, a small corner patch is placed at the bottom-left corner 
    so there is no extra unfilled square.

    Parameters:
      multihead_attention (torch.Tensor): shape (1, num_heads, n_tokens, n_tokens).
      output_path (str): File path to save the figure.
      title (str): Plot title.
      vision_idx (dict or None): Dictionary with 'start' and 'end' for vision tokens.
      show_image (bool): If True, display the plot.
      pool_stride (int): Stride used for average pooling.
      dpi (int): Dots per inch for the figure.
    
    Returns:
      top_ten_attentions (list): A list with top 10 (index, score) pairs for each row.
      averaged_attention (torch.Tensor): The 2D pooled attention map.
    """
    print("multihead_attention shape:", multihead_attention.shape)
    
    # 1) Average attention scores over heads
    averaged_attention = torch.mean(multihead_attention, axis=1)[0].float()

    # 2) Downsample via average pooling
    averaged_attention = torch.nn.functional.avg_pool2d(
        averaged_attention.unsqueeze(0).unsqueeze(0), 
        kernel_size=pool_stride, 
        stride=pool_stride
    ).squeeze(0).squeeze(0)

    # Number of pooled tokens (square shape)
    n_cells = averaged_attention.shape[0]

    # 3) Create figure & main heatmap axis (no tick labels)
    fig, ax = plt.subplots(figsize=(10, 10), dpi=dpi)
    
    cmap = plt.get_cmap("inferno")
    log_norm = LogNorm(vmin=0.0007, vmax=0.1)
    
    hm = sns.heatmap(
        averaged_attention,
        cmap=cmap,
        norm=log_norm,
        cbar_kws={'label': 'Attention score'},
        ax=ax,
        xticklabels=False,
        yticklabels=False
    )
    
    if vision_idx is not None:
        # Convert original token indices → pooled indices
        pooled_v_start = vision_idx['start'] // pool_stride
        pooled_v_end   = vision_idx['end']   // pool_stride
        
        # 0 => text token, 1 => vision token
        token_types = np.zeros(n_cells, dtype=int)
        token_types[pooled_v_start:pooled_v_end] = 1
        
        # For the strips
        left_strip   = token_types.reshape(n_cells, 1)  # vertical
        bottom_strip = token_types.reshape(1, n_cells)  # horizontal

        # Colormap for the strips: 0=blue, 1=red
        token_cmap = ListedColormap(['lightblue', 'lightsalmon'])
        
        # 4) Position these strips flush with the heatmap
        pos = ax.get_position()

        # Thinner bars
        left_bar_width    = 0.02
        bottom_bar_height = 0.02

        # Left strip
        left_ax = fig.add_axes([
            pos.x0 - left_bar_width - 0.01,
            pos.y0,
            left_bar_width ,
            pos.height
        ])
        left_ax.imshow(left_strip, aspect='auto', cmap=token_cmap)
        left_ax.axis('off')

        # Bottom strip
        bottom_ax = fig.add_axes([
            pos.x0,
            pos.y0 - bottom_bar_height - 0.01,
            pos.width,
            bottom_bar_height
        ])
        bottom_ax.imshow(bottom_strip, aspect='auto', cmap=token_cmap)
        bottom_ax.axis('off')
        
        # 5) Corner patch
        #   Fill the small square corner at bottom-left so there is no gap.
        corner_ax = fig.add_axes([
            pos.x0 - left_bar_width,           # left strip's x
            pos.y0 - bottom_bar_height,        # bottom strip's y
            left_bar_width,                    # same width as left strip
            bottom_bar_height                  # same height as bottom strip
        ])
        # Decide what color to fill the corner with:
        # Typically, if your text region covers index=0, we fill with text (blue).
        # If your vision region starts at 0, you might want red. 
        # For simplicity, let's assume text color:
        corner_ax.set_facecolor('blue')
        corner_ax.axis('off')
        
        # Legend: Blue patch for text, red patch for vision
        blue_patch = mpatches.Patch(color='lightblue', label='Text tokens (System+Instruction)')
        red_patch  = mpatches.Patch(color='lightsalmon',  label='Vision tokens')
        fig.legend(
            handles=[blue_patch, red_patch],
            loc='lower center',
            ncol=2,
            fontsize=12,
            bbox_to_anchor=(0.5, 0.02)
        )
        # Adjust bottom margin to fit the legend
        fig.subplots_adjust(bottom=0.11)
    
    # Increase title & colorbar font sizes
    ax.set_title(title, fontsize=16)
    cbar = hm.collections[0].colorbar
    cbar.ax.set_ylabel('Attention Score', fontsize=14)
    cbar.ax.tick_params(labelsize=12)

    plt.savefig(output_path, bbox_inches='tight')
    if show_image:
        plt.show()
    plt.close(fig)

    # Gather top 10 attentions for each row
    top_ten_attentions = []
    for row in averaged_attention:
        top_vals, top_idx = torch.topk(row, 10)
        top_line = list(zip(top_idx.tolist(), top_vals.tolist()))
        top_ten_attentions.append(top_line)
    
    return top_ten_attentions, averaged_attention

# Example usage:
# multihead_attention = ...  # shape (1, num_heads, n_tokens, n_tokens)
# vision_idx = {'start': 500, 'end': 600}
# visualize_attention(multihead_attention, output_path="atten_map_1.png",
#                     title="Layer 5", vision_idx=vision_idx, show_image=True)


In [30]:
for i, attention in enumerate(attention_outputs[0]):
    # print(i, attention)
    # print(top5_attention)
    top5_attention,average_attentions = visualize_attention(attention.cpu(), output_path="./attn_maps/atten_map_"+str(i+1)+".png",title="Layer "+str(i+1), vision_idx=vision_idx, show_image=False)
    # if i == 0:
    #     break

multihead_attention shape: torch.Size([1, 28, 1202, 1202])
multihead_attention shape: torch.Size([1, 28, 1202, 1202])
multihead_attention shape: torch.Size([1, 28, 1202, 1202])
multihead_attention shape: torch.Size([1, 28, 1202, 1202])
multihead_attention shape: torch.Size([1, 28, 1202, 1202])
multihead_attention shape: torch.Size([1, 28, 1202, 1202])
multihead_attention shape: torch.Size([1, 28, 1202, 1202])
multihead_attention shape: torch.Size([1, 28, 1202, 1202])
multihead_attention shape: torch.Size([1, 28, 1202, 1202])
multihead_attention shape: torch.Size([1, 28, 1202, 1202])
multihead_attention shape: torch.Size([1, 28, 1202, 1202])
multihead_attention shape: torch.Size([1, 28, 1202, 1202])
multihead_attention shape: torch.Size([1, 28, 1202, 1202])
multihead_attention shape: torch.Size([1, 28, 1202, 1202])
multihead_attention shape: torch.Size([1, 28, 1202, 1202])
multihead_attention shape: torch.Size([1, 28, 1202, 1202])
multihead_attention shape: torch.Size([1, 28, 1202, 1202

## Global (All attention in 28 layers in one picture)

In [35]:
def visualize_global_attention(attention_list, output_path="global_atten.png", 
                               pool_stride=20, vision_idx=None, show_image=False):
    """
    Plots all attention maps (one per layer) in one figure without any tick marks.
    
    Parameters:
      attention_list (list[torch.Tensor]): A list of attention tensors (each with shape: (1, num_heads, n_tokens, n_tokens)).
      output_path (str): Path to save the resulting global image.
      pool_stride (int): Stride value for average pooling.
      vision_idx (dict or None): Optional dictionary with 'start' and 'end' keys to highlight vision tokens.
      show_image (bool): Whether to call plt.show() to display the image.
      
    Returns:
      None (the global figure is saved to output_path).
    """

    import torch
    import seaborn as sns
    import matplotlib.pyplot as plt
    from matplotlib.colors import LogNorm

    n_layers = len(attention_list)
    # Define grid dimensions; for 28 layers, 7 rows x 4 columns works nicely.
    ncols = 4
    nrows = (n_layers + ncols - 1) // ncols  # rounds up if not a perfect fit

    # Create the figure and axes.
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, 
                             figsize=(4*ncols, 4*nrows), dpi=200)
    # Flatten axes array to simplify iteration.
    axes = axes.flatten()

    # Define colormap and normalization (same settings as before).
    cmap = plt.get_cmap("inferno")
    log_norm = LogNorm(vmin=0.0007, vmax=0.1)

    # Process and plot each layer's attention.
    for i, multihead_attention in enumerate(attention_list):
        ax = axes[i]
        # Average over heads. Expecting shape: (1, num_heads, n_tokens, n_tokens)
        averaged_attention = torch.mean(multihead_attention, axis=1)[0].float()

        # Apply average pooling (note: avg_pool2d requires a 4D tensor).
        averaged_attention = torch.nn.functional.avg_pool2d(
            averaged_attention.unsqueeze(0).unsqueeze(0),
            pool_stride,
            stride=pool_stride
        ).squeeze(0).squeeze(0)

        # Plot heatmap on the specified axis without an individual colorbar.
        sns.heatmap(
            averaged_attention,
            ax=ax,
            cmap=cmap,
            norm=log_norm,
            cbar=False
        )

        # Remove all tick marks.
        ax.set_xticks([])
        ax.set_yticks([])

        # Set a title for each subplot.
        ax.set_title(f"Layer {i+1}", fontsize=8)

    # Remove any empty subplots if n_layers does not exactly fill the grid.
    for j in range(n_layers, nrows * ncols):
        fig.delaxes(axes[j])

    # Add a single global colorbar on the right side.
    fig.subplots_adjust(right=0.85)
    cbar_ax = fig.add_axes([0.9, 0.15, 0.03, 0.7])
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=log_norm)
    sm.set_array([])
    fig.colorbar(sm, cax=cbar_ax, label='Attention score')

    # Instead of using tight_layout (which can give warnings when using extra axes),
    # we manually adjust the subplot layout.
    # plt.tight_layout(rect=[0, 0, 0.9, 1])

    # Save and optionally display the figure.
    plt.savefig(output_path, bbox_inches='tight')
    if show_image:
        plt.show()
    plt.close()


In [ ]:
attention_output_cpu = [attention_outputs[0][i].cpu() for i in range(28)]
visualize_global_attention(attention_output_cpu, output_path="./attn_maps/global_atten_map.png", vision_idx=vision_idx, show_image=True)
